In [1]:
import os
import struct
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
from PIL import Image

class TMDProcessor:
    """
    A class to process TMD files. Supports reading TMD files generated by TrueMap v6
    or GelSight (with a slightly different layout), visualizing the height map in various ways,
    and converting the height data to a displacement image or an STL file.
    """
    # Class constants
    MAX_PIXELS = 1e8           # Maximum allowed product of width * height
    COLORBAR_LABEL = "Height (µm)"
    SCALE_FACTORS = [0.5, 1, 2, 3]  # Z-axis scaling factors for slider
    
    def __init__(self, file_path):
        """
        Initialize with the given TMD file path.
        """
        self.file_path = file_path
        self.basename = os.path.basename(file_path)
        self.data = None  # Will hold parsed data
    
    @staticmethod
    def print_file_header(file_path, num_bytes=64):
        """
        Prints the first `num_bytes` of the file in hex and ASCII.
        """
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")
        with open(file_path, 'rb') as f:
            header_bytes = f.read(num_bytes)
        print("File header (hex):", header_bytes.hex())
        print("File header (ASCII):", header_bytes.decode('ascii', errors='replace'))
    
    @staticmethod
    def read_null_terminated_string(f, chunk_size=256):
        """
        Reads a null-terminated ASCII string from the file.
        Reads up to `chunk_size` bytes, finds the null, and resets the file pointer to just after it.
        """
        pos = f.tell()
        chunk = f.read(chunk_size)
        null_index = chunk.find(b'\x00')
        if null_index == -1:
            return chunk.decode('ascii', errors='ignore')
        else:
            f.seek(pos + null_index + 1)
            return chunk[:null_index].decode('ascii', errors='ignore')
    
    @classmethod
    def read_tmd_endian(cls, file_path, endian):
        """
        Reads a TrueMap-like TMD file using the given endian format ('<' for little-endian,
        '>' for big-endian). Returns a dictionary with metadata and a 2D NumPy array for the height map.
        """
        with open(file_path, 'rb') as f:
            header = f.read(32).decode('ascii', errors='ignore')
            if "Binary TrueMap Data File" not in header:
                print(f"Warning: Unexpected header: {header.strip()}")
            comment = cls.read_null_terminated_string(f, chunk_size=256)
            width, height = struct.unpack(f'{endian}II', f.read(8))
            if width * height > cls.MAX_PIXELS:
                raise ValueError(f"Dimensions too large: width={width}, height={height} (product={width*height})")
            x_length, y_length, x_offset, y_offset = struct.unpack(f'{endian}ffff', f.read(16))
            num_points = width * height
            height_bytes = f.read(num_points * 4)
            if len(height_bytes) != num_points * 4:
                raise ValueError("File ended unexpectedly while reading height data.")
            height_data = np.frombuffer(height_bytes, dtype=np.float32)
            height_map = height_data.reshape((height, width))
        return {
            'header': header,
            'comment': comment,
            'width': width,
            'height': height,
            'x_length': x_length,
            'y_length': y_length,
            'x_offset': x_offset,
            'y_offset': y_offset,
            'height_map': height_map
        }
    
    @classmethod
    def read_tmd_auto(cls, file_path, threshold=None):
        """
        Attempts to read the TMD file using both little- and big-endian interpretations.
        Returns the parsed data whose product (width*height) is below `threshold` (or the one with the smaller product).
        """
        if threshold is None:
            threshold = cls.MAX_PIXELS
        print("Trying little-endian interpretation...")
        data_le = None
        try:
            data_le = cls.read_tmd_endian(file_path, '<')
            product_le = data_le['width'] * data_le['height']
            print(f"Little-endian dims: {data_le['width']} x {data_le['height']} (product={product_le})")
        except Exception as e:
            print("Little-endian failed:", e)
        print("Trying big-endian interpretation...")
        data_be = None
        try:
            data_be = cls.read_tmd_endian(file_path, '>')
            product_be = data_be['width'] * data_be['height']
            print(f"Big-endian dims: {data_be['width']} x {data_be['height']} (product={product_be})")
        except Exception as e:
            print("Big-endian failed:", e)
        if data_le and data_le['width'] * data_le['height'] <= threshold:
            print("Using little-endian interpretation.\n")
            return data_le
        elif data_be and data_be['width'] * data_be['height'] <= threshold:
            print("Using big-endian interpretation.\n")
            return data_be
        else:
            if data_le and data_be:
                if data_le['width'] * data_le['height'] < data_be['width'] * data_be['height']:
                    print("Neither below threshold; using little-endian (smaller product).\n")
                    return data_le
                else:
                    print("Neither below threshold; using big-endian (smaller product).\n")
                    return data_be
            elif data_le:
                return data_le
            elif data_be:
                return data_be
            else:
                raise ValueError("Could not read the TMD file with either endianness.")
    
    @staticmethod
    def read_gelsight_tmd(file_path):
        """
        Reads a GelSight-generated TMD file.
        Assumes little-endian and a similar layout as TrueMap.
        """
        with open(file_path, 'rb') as f:
            header = f.read(32).decode('ascii', errors='ignore')
            comment = TMDProcessor.read_null_terminated_string(f, chunk_size=256)
            width, height = struct.unpack('<II', f.read(8))
            if width * height > TMDProcessor.MAX_PIXELS:
                raise ValueError(f"Dimensions too large: width={width}, height={height} (product={width*height})")
            x_length, y_length, x_offset, y_offset = struct.unpack('<ffff', f.read(16))
            num_points = width * height
            height_bytes = f.read(num_points * 4)
            if len(height_bytes) != num_points * 4:
                raise ValueError("File ended unexpectedly while reading height data.")
            height_data = np.frombuffer(height_bytes, dtype=np.float32)
            height_map = height_data.reshape((height, width))
        return {
            'header': header,
            'comment': comment,
            'width': width,
            'height': height,
            'x_length': x_length,
            'y_length': y_length,
            'x_offset': x_offset,
            'y_offset': y_offset,
            'height_map': height_map
        }
    
    @classmethod
    def plot_height_map_with_slider(cls, height_map, colorbar_label=None, html_filename="slider_plot.html", partial_range=None):
        """
        Creates a 3D surface plot of the height map with a color bar (labeled by colorbar_label)
        and a slider to adjust the vertical scaling (Z-axis aspect ratio).
        Optionally, only a partial region of the height map is rendered if partial_range is provided.
        """
        if colorbar_label is None:
            colorbar_label = cls.COLORBAR_LABEL
        if partial_range is not None:
            height_map = height_map[partial_range[0]:partial_range[1], partial_range[2]:partial_range[3]]
            print(f"Partial render applied: rows {partial_range[0]}:{partial_range[1]}, cols {partial_range[2]}:{partial_range[3]}")
        zmin = float(height_map.min())
        zmax = float(height_map.max())
        surface = go.Surface(
            z=height_map,
            cmin=zmin,
            cmax=zmax,
            colorscale='Viridis',
            colorbar=dict(title=colorbar_label)
        )
        fig = go.Figure(data=[surface])
        fig.update_layout(
            title='3D Surface Plot',
            scene=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title=colorbar_label,
                aspectmode='cube'
            ),
            margin=dict(l=65, r=50, b=65, t=90)
        )
        steps = []
        for sf in cls.SCALE_FACTORS:
            steps.append(dict(
                method='relayout',
                args=[{'scene.aspectratio': dict(x=1, y=1, z=sf)}],
                label=f"{sf}x"
            ))
        sliders = [dict(
            active=1,
            currentvalue={"prefix": "Z-scale: "},
            steps=steps,
            pad={"t": 50}
        )]
        fig.update_layout(sliders=sliders)
        fig.show()
        fig.write_html(html_filename, include_plotlyjs="cdn")
        print(f"3D Plot saved to {html_filename}")
        return fig
    
    @classmethod
    def plot_2d_heatmap(cls, height_map, colorbar_label=None, html_filename="2d_heatmap.html"):
        """
        Creates a 2D heatmap of the complete height map and saves it as an HTML file.
        """
        if colorbar_label is None:
            colorbar_label = cls.COLORBAR_LABEL
        fig = go.Figure(data=go.Heatmap(
            z=height_map,
            colorscale='Viridis',
            colorbar=dict(title=colorbar_label)
        ))
        fig.update_layout(
            title="2D Heatmap of Complete Height Map",
            xaxis_title="X",
            yaxis_title="Y"
        )
        fig.show()
        fig.write_html(html_filename, include_plotlyjs="cdn")
        print(f"2D Heatmap saved to {html_filename}")
        return fig
    
    @classmethod
    def extract_and_plot_x_profile(cls, data, profile_row=None, html_filename="x_profile.html"):
        """
        Extracts an X profile (default: middle row) from the height map, prints the first 10 x coordinates and heights,
        and plots a 2D line chart.
        """
        height_map = data['height_map']
        width = data['width']
        if profile_row is None:
            profile_row = height_map.shape[0] // 2
        x_coords = np.linspace(data['x_offset'], data['x_offset'] + data['x_length'], num=width)
        x_profile = height_map[profile_row, :]
        print(f"\nX Profile at row {profile_row}:")
        print("X coordinates (first 10):", x_coords[:10])
        print("Heights (first 10):", x_profile[:10])
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x_coords, y=x_profile, mode='lines+markers', name='X Profile'))
        fig.update_layout(
            title=f"X Profile (row {profile_row})",
            xaxis_title='X Coordinate',
            yaxis_title=cls.COLORBAR_LABEL
        )
        fig.show()
        fig.write_html(html_filename, include_plotlyjs="cdn")
        print(f"X Profile plot saved to {html_filename}")
        return x_coords, x_profile
    
    @staticmethod
    def convert_heightmap_to_displacement_map(height_map, filename="displacement_map.png"):
        """
        Converts the full height map into a displacement map image (grayscale PNG).
        The height values are normalized to 0-255.
        """
        hmin = float(height_map.min())
        hmax = float(height_map.max())
        norm = (height_map - hmin) / (hmax - hmin) * 255.0
        norm = norm.astype(np.uint8)
        im = Image.fromarray(norm)
        im.save(filename)
        print(f"Displacement map saved to {filename}")
    
    @staticmethod
    def convert_heightmap_to_stl(height_map, filename="output.stl", x_offset=0, y_offset=0, x_length=1, y_length=1, z_scale=1):
        """
        Converts the full height map into an ASCII STL file.
        Uses a grid approach where each grid cell is split into two triangles.
        """
        rows, cols = height_map.shape
        if cols < 2 or rows < 2:
            print("Height map too small to generate STL.")
            return
        x_scale = x_length / (cols - 1)
        y_scale = y_length / (rows - 1)
        vertices = np.zeros((rows, cols, 3))
        for i in range(rows):
            for j in range(cols):
                vertices[i, j] = [x_offset + j * x_scale, y_offset + i * y_scale, height_map[i, j] * z_scale]
        triangles = []
        for i in range(rows - 1):
            for j in range(cols - 1):
                v0 = vertices[i, j]
                v1 = vertices[i, j+1]
                v2 = vertices[i+1, j+1]
                v3 = vertices[i+1, j]
                triangles.append((v0, v1, v2))
                triangles.append((v0, v2, v3))
        with open(filename, "w") as f:
            f.write("solid displacement\n")
            for tri in triangles:
                v0, v1, v2 = tri
                n = np.cross(v1 - v0, v2 - v0)
                norm_val = np.linalg.norm(n)
                if norm_val == 0:
                    n = np.array([0, 0, 0])
                else:
                    n = n / norm_val
                f.write(f"  facet normal {n[0]:.6e} {n[1]:.6e} {n[2]:.6e}\n")
                f.write("    outer loop\n")
                for vertex in tri:
                    f.write(f"      vertex {vertex[0]:.6e} {vertex[1]:.6e} {vertex[2]:.6e}\n")
                f.write("    endloop\n")
                f.write("  endfacet\n")
            f.write("endsolid displacement\n")
        print(f"STL file saved to {filename}")
    
    def process(self, partial_range=None):
        """
        Processes the TMD file by:
          - Inspecting the header.
          - Reading the data using the appropriate parser.
          - Printing metadata.
          - Creating a 3D surface plot with slider (with optional partial rendering).
          - Creating a complete 2D heatmap.
          - Extracting and plotting an X profile.
          - Converting the height map to a displacement map image.
          - Converting the height map to an STL file.
        """
        print("=" * 80)
        print(f"Processing file: {self.file_path}\n")
        # Inspect header
        try:
            self.print_file_header(self.file_path, num_bytes=64)
        except Exception as e:
            print(f"Error inspecting header: {e}")
            return
        # Determine parser by filename
        if self.basename == "Dime.tmd":
            print("\nParsing as TrueMap v6...")
            try:
                self.data = self.read_tmd_auto(self.file_path)
            except Exception as e:
                print(f"Error reading file (TrueMap parser): {e}")
                return
        elif self.basename == "circle_0mm_100g_heightmap_linear_detrend.tmd":
            print("\nParsing as GelSight TMD...")
            try:
                self.data = self.read_gelsight_tmd(self.file_path)
            except Exception as e:
                print(f"Error reading file (GelSight parser): {e}")
                return
        else:
            print("Unknown file format. Aborting.")
            return
        # Print metadata
        print("\nSuccessfully read the file.")
        print("Header:", self.data['header'])
        print("Comment:", self.data['comment'])
        print(f"Dimensions: {self.data['width']} x {self.data['height']}")
        print(f"X length: {self.data['x_length']}, Y length: {self.data['y_length']}")
        print(f"X offset: {self.data['x_offset']}, Y offset: {self.data['y_offset']}")
        # Create output file names based on base name
        base_name = os.path.splitext(self.basename)[0]
        slider_html = f"{base_name}_3D_slider.html"
        heatmap_html = f"{base_name}_2D_heatmap.html"
        x_profile_html = f"{base_name}_x_profile.html"
        displacement_img = f"{base_name}_displacement.png"
        stl_filename = f"{base_name}.stl"
        # Create 3D surface plot with slider (apply partial_range if provided)
        self.plot_height_map_with_slider(self.data['height_map'], html_filename=slider_html, partial_range=partial_range)
        # Create 2D heatmap of complete height map
        self.plot_2d_heatmap(self.data['height_map'], html_filename=heatmap_html)
        # Extract and plot X profile
        self.extract_and_plot_x_profile(self.data, html_filename=x_profile_html)
        # Convert full height map to displacement map image
        self.convert_heightmap_to_displacement_map(self.data['height_map'], filename=displacement_img)
        # Convert full height map to STL file
        self.convert_heightmap_to_stl(
            self.data['height_map'],
            filename=stl_filename,
            x_offset=self.data['x_offset'],
            y_offset=self.data['y_offset'],
            x_length=self.data['x_length'],
            y_length=self.data['y_length'],
            z_scale=1  # Adjust z_scale as needed
        )
        print("=" * 80, "\n")
        
###############################################################################
#                           Main Execution                                  #
###############################################################################
if __name__ == "__main__":
    # Set Plotly renderer if needed
    pio.renderers.default = "browser"
    
    # List of TMD files to process
    file_paths = [
        "Dime.tmd",
        "circle_0mm_100g_heightmap_linear_detrend.tmd"
    ]
    # For the "circle" file, specify a partial render region if desired (e.g., central region)
    partial_range_circle = (50, 250, 50, 250)  # Adjust as needed
    
    for fp in file_paths:
        processor = TMDProcessor(fp)
        if os.path.basename(fp) == "circle_0mm_100g_heightmap_linear_detrend.tmd":
            processor.process(partial_range=partial_range_circle)
        else:
            processor.process()

Processing file: Dime.tmd

File header (hex): 42696e61727920547275654d617020446174612046696c652076322e300d0a004372656174656420627920547275654d61702076360d0a002c0100002c010000
File header (ASCII): Binary TrueMap Data File v2.0
 Created by TrueMap v6
 ,  ,  

Parsing as TrueMap v6...
Trying little-endian interpretation...
Little-endian dims: 300 x 300 (product=90000)
Trying big-endian interpretation...
Big-endian failed: Dimensions too large: width=738263040, height=738263040 (product=545032316230041600)
Using little-endian interpretation.


Successfully read the file.
Header: Binary TrueMap Data File v2.0
 
Comment: Created by TrueMap v6

Dimensions: 300 x 300
X length: 18.956600189208984, Y length: 18.956600189208984
X offset: 0.0, Y offset: 0.0
3D Plot saved to Dime_3D_slider.html
2D Heatmap saved to Dime_2D_heatmap.html

X Profile at row 150:
X coordinates (first 10): [0.         0.0634     0.1268     0.1902     0.2536     0.317
 0.3804     0.4438     0.50720001 0.57060001]
Heights